In [1]:
from config import CLIENT_ID, CLIENT_SECRET
import os
import pprint
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
playlist_df = pd.read_csv(f'./Resources/playlists.csv')

In [3]:
playlist_df = playlist_df.dropna(axis=0)

In [4]:
playlist_df

,Playlist,Followers,Songs,Genre 1,Genre 2,URI
0,Today’s Top Hits,"31,728,611",50,Pop,R&B,spotify:playlist:37i9dQZF1DXcBWIGoYBM5M
1,Feel Good Indie Rock,"1,664,372",120,Rock,Indie,spotify:playlist:37i9dQZF1DX2sUQwD7tbmL
2,Indie Rock Road Trip,"895,126",100,Rock,Indie,spotify:playlist:37i9dQZF1DWUoqEG4WY6ce
3,Rock N' Run,"490,726",50,Rock,Workout,spotify:playlist:37i9dQZF1DWXx3Txis2L4x
4,Rock This,"4,649,112",50,Rock,Alternative,spotify:playlist:37i9dQZF1DXcF6B6QPhFDv
...,...,...,...,...,...,...
95,Beats & Rhymes,"403,154",100,Rap,K-Pop,spotify:playlist:37i9dQZF1DXcA6dRp8rwj6
96,The New Alt,"1,095,705",125,Alternative,Rock,spotify:playlist:37i9dQZF1DX82GYcclJ3Ug
97,Pop Punk's Not Dead,"523,146",80,Rock,Pop,spotify:playlist:37i9dQZF1DX1ewVhAJ17m4
98,Signed XOXO,"1,912,124",98,R&B,Pop,spotify:playlist:37i9dQZF1DX2A29LI7xHn1


In [5]:
# Set environment variables
os.environ['SPOTIPY_CLIENT_ID'] = CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET'] = CLIENT_SECRET

In [6]:
# instantiate Spotify class
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [ ]:
#playlist_dfs = []
songs_dfs = []

# Iterate over playlist dataframe to grab song info for each playlist
for index, row in playlist_df[:50].iterrows():
    
    print(f'Processing playlist {index} of {len(playlist_df)}')
    
    # Grabbing URI for the playlist
    playlist_uri = row['URI'] 
    print(playlist_uri)
    
    # Make temporary table for each playlist: subsetting playlist df
    temp = playlist_df[playlist_df['URI']==playlist_uri]
    
    # Get all songs from playlist
    song_count = len(sp.playlist_tracks(playlist_uri)["items"])
    
    # Temporary table for all songs in this playlist
    songs = []
    
    # Get info for each song
    for song_index in range(0, song_count):

        print(f'\t Adding info for song {song_index} out of {song_count}... \n')
        
        song_temp = temp.copy()
        
        ## Add song info to the table
        # Grab URI
        song_temp['Song URI'] = sp.playlist_tracks(playlist_uri)["items"][song_index]["track"]["uri"]
        
        # Grab Popularity
        song_temp['Popularity'] = sp.playlist_tracks(playlist_uri)["items"][song_index]["track"]["popularity"]
        
        # Grab Artist Name
        song_temp['Artist Name'] = sp.playlist_tracks(playlist_uri)["items"][song_index]["track"]["artists"][0]['name']
        
        # Grab Song Name
        song_temp['Song Name'] = sp.playlist_tracks(playlist_uri)["items"][song_index]["track"]["name"]
        
        # Grab popularity score
        songs.append(song_temp)
        
    
    # Make song table
    songs = pd.concat(songs)
    
    # Add to playlist
    #playlist_dfs.append(songs)
    songs_dfs.append(songs)

# Bring everything together 
pd.concat(songs_dfs)
#pd.concat(playlist_dfs)

Processing playlist 0 of 100
spotify:playlist:37i9dQZF1DXcBWIGoYBM5M
	 Adding info for song 0 out of 50... 

	 Adding info for song 1 out of 50... 

	 Adding info for song 2 out of 50... 

	 Adding info for song 3 out of 50... 

	 Adding info for song 4 out of 50... 

	 Adding info for song 5 out of 50... 

	 Adding info for song 6 out of 50... 

	 Adding info for song 7 out of 50... 

	 Adding info for song 8 out of 50... 

	 Adding info for song 9 out of 50... 

	 Adding info for song 10 out of 50... 

	 Adding info for song 11 out of 50... 

	 Adding info for song 12 out of 50... 

	 Adding info for song 13 out of 50... 

	 Adding info for song 14 out of 50... 

	 Adding info for song 15 out of 50... 

	 Adding info for song 16 out of 50... 

	 Adding info for song 17 out of 50... 

	 Adding info for song 18 out of 50... 

	 Adding info for song 19 out of 50... 

	 Adding info for song 20 out of 50... 

	 Adding info for song 21 out of 50... 

	 Adding info for song 22 out of 50... 

	 Adding info for song 43 out of 100... 

	 Adding info for song 44 out of 100... 

	 Adding info for song 45 out of 100... 

	 Adding info for song 46 out of 100... 

	 Adding info for song 47 out of 100... 

	 Adding info for song 48 out of 100... 

	 Adding info for song 49 out of 100... 

	 Adding info for song 50 out of 100... 

	 Adding info for song 51 out of 100... 

	 Adding info for song 52 out of 100... 

	 Adding info for song 53 out of 100... 

	 Adding info for song 54 out of 100... 

	 Adding info for song 55 out of 100... 

	 Adding info for song 56 out of 100... 

	 Adding info for song 57 out of 100... 

	 Adding info for song 58 out of 100... 

	 Adding info for song 59 out of 100... 

	 Adding info for song 60 out of 100... 

	 Adding info for song 61 out of 100... 

	 Adding info for song 62 out of 100... 

	 Adding info for song 63 out of 100... 

	 Adding info for song 64 out of 100... 

	 Adding info for song 65 out of 100... 

	 Adding info for song 66 out of 1

	 Adding info for song 38 out of 50... 

	 Adding info for song 39 out of 50... 

	 Adding info for song 40 out of 50... 

	 Adding info for song 41 out of 50... 

	 Adding info for song 42 out of 50... 

	 Adding info for song 43 out of 50... 

	 Adding info for song 44 out of 50... 

	 Adding info for song 45 out of 50... 

	 Adding info for song 46 out of 50... 

	 Adding info for song 47 out of 50... 

	 Adding info for song 48 out of 50... 

	 Adding info for song 49 out of 50... 

Processing playlist 6 of 100
spotify:playlist:37i9dQZF1DWWJOmJ7nRx0C
	 Adding info for song 0 out of 100... 

	 Adding info for song 1 out of 100... 

	 Adding info for song 2 out of 100... 

	 Adding info for song 3 out of 100... 

	 Adding info for song 4 out of 100... 

	 Adding info for song 5 out of 100... 

	 Adding info for song 6 out of 100... 

	 Adding info for song 7 out of 100... 

	 Adding info for song 8 out of 100... 

	 Adding info for song 9 out of 100... 

	 Adding info for song 10 out

	 Adding info for song 26 out of 100... 

	 Adding info for song 27 out of 100... 

	 Adding info for song 28 out of 100... 

	 Adding info for song 29 out of 100... 

	 Adding info for song 30 out of 100... 

	 Adding info for song 31 out of 100... 

	 Adding info for song 32 out of 100... 

	 Adding info for song 33 out of 100... 

	 Adding info for song 34 out of 100... 

	 Adding info for song 35 out of 100... 

	 Adding info for song 36 out of 100... 

	 Adding info for song 37 out of 100... 

	 Adding info for song 38 out of 100... 

	 Adding info for song 39 out of 100... 

	 Adding info for song 40 out of 100... 

	 Adding info for song 41 out of 100... 

	 Adding info for song 42 out of 100... 

	 Adding info for song 43 out of 100... 

	 Adding info for song 44 out of 100... 

	 Adding info for song 45 out of 100... 

	 Adding info for song 46 out of 100... 

	 Adding info for song 47 out of 100... 

	 Adding info for song 48 out of 100... 

	 Adding info for song 49 out of 1

	 Adding info for song 19 out of 51... 

	 Adding info for song 20 out of 51... 

	 Adding info for song 21 out of 51... 

	 Adding info for song 22 out of 51... 

	 Adding info for song 23 out of 51... 

	 Adding info for song 24 out of 51... 

	 Adding info for song 25 out of 51... 

	 Adding info for song 26 out of 51... 

	 Adding info for song 27 out of 51... 

	 Adding info for song 28 out of 51... 

	 Adding info for song 29 out of 51... 

	 Adding info for song 30 out of 51... 

	 Adding info for song 31 out of 51... 

	 Adding info for song 32 out of 51... 

	 Adding info for song 33 out of 51... 

	 Adding info for song 34 out of 51... 

	 Adding info for song 35 out of 51... 

	 Adding info for song 36 out of 51... 

	 Adding info for song 37 out of 51... 

	 Adding info for song 38 out of 51... 

	 Adding info for song 39 out of 51... 

	 Adding info for song 40 out of 51... 

	 Adding info for song 41 out of 51... 

	 Adding info for song 42 out of 51... 

	 Adding info fo

	 Adding info for song 11 out of 87... 

	 Adding info for song 12 out of 87... 

	 Adding info for song 13 out of 87... 

	 Adding info for song 14 out of 87... 

	 Adding info for song 15 out of 87... 

	 Adding info for song 16 out of 87... 

	 Adding info for song 17 out of 87... 

	 Adding info for song 18 out of 87... 

	 Adding info for song 19 out of 87... 

	 Adding info for song 20 out of 87... 

	 Adding info for song 21 out of 87... 

	 Adding info for song 22 out of 87... 

	 Adding info for song 23 out of 87... 

	 Adding info for song 24 out of 87... 

	 Adding info for song 25 out of 87... 

	 Adding info for song 26 out of 87... 

	 Adding info for song 27 out of 87... 

	 Adding info for song 28 out of 87... 

	 Adding info for song 29 out of 87... 

	 Adding info for song 30 out of 87... 

	 Adding info for song 31 out of 87... 

	 Adding info for song 32 out of 87... 

	 Adding info for song 33 out of 87... 

	 Adding info for song 34 out of 87... 

	 Adding info fo

	 Adding info for song 19 out of 75... 

	 Adding info for song 20 out of 75... 

	 Adding info for song 21 out of 75... 

	 Adding info for song 22 out of 75... 

	 Adding info for song 23 out of 75... 

	 Adding info for song 24 out of 75... 

	 Adding info for song 25 out of 75... 

	 Adding info for song 26 out of 75... 

	 Adding info for song 27 out of 75... 

	 Adding info for song 28 out of 75... 

	 Adding info for song 29 out of 75... 

	 Adding info for song 30 out of 75... 

	 Adding info for song 31 out of 75... 

	 Adding info for song 32 out of 75... 

	 Adding info for song 33 out of 75... 

	 Adding info for song 34 out of 75... 

	 Adding info for song 35 out of 75... 

	 Adding info for song 36 out of 75... 

	 Adding info for song 37 out of 75... 

	 Adding info for song 38 out of 75... 

	 Adding info for song 39 out of 75... 

	 Adding info for song 40 out of 75... 

	 Adding info for song 41 out of 75... 

	 Adding info for song 42 out of 75... 

	 Adding info fo

	 Adding info for song 0 out of 75... 

	 Adding info for song 1 out of 75... 

	 Adding info for song 2 out of 75... 

	 Adding info for song 3 out of 75... 

	 Adding info for song 4 out of 75... 

	 Adding info for song 5 out of 75... 

	 Adding info for song 6 out of 75... 

	 Adding info for song 7 out of 75... 

	 Adding info for song 8 out of 75... 

	 Adding info for song 9 out of 75... 

	 Adding info for song 10 out of 75... 

	 Adding info for song 11 out of 75... 

	 Adding info for song 12 out of 75... 

	 Adding info for song 13 out of 75... 

	 Adding info for song 14 out of 75... 

	 Adding info for song 15 out of 75... 

	 Adding info for song 16 out of 75... 

	 Adding info for song 17 out of 75... 

	 Adding info for song 18 out of 75... 

	 Adding info for song 19 out of 75... 

	 Adding info for song 20 out of 75... 

	 Adding info for song 21 out of 75... 

	 Adding info for song 22 out of 75... 

	 Adding info for song 23 out of 75... 

	 Adding info for song 24 

	 Adding info for song 20 out of 100... 

	 Adding info for song 21 out of 100... 

	 Adding info for song 22 out of 100... 

	 Adding info for song 23 out of 100... 

	 Adding info for song 24 out of 100... 

	 Adding info for song 25 out of 100... 

	 Adding info for song 26 out of 100... 

	 Adding info for song 27 out of 100... 

	 Adding info for song 28 out of 100... 

	 Adding info for song 29 out of 100... 

	 Adding info for song 30 out of 100... 

	 Adding info for song 31 out of 100... 

	 Adding info for song 32 out of 100... 

	 Adding info for song 33 out of 100... 

	 Adding info for song 34 out of 100... 

	 Adding info for song 35 out of 100... 

	 Adding info for song 36 out of 100... 

	 Adding info for song 37 out of 100... 

	 Adding info for song 38 out of 100... 

	 Adding info for song 39 out of 100... 

	 Adding info for song 40 out of 100... 

	 Adding info for song 41 out of 100... 

	 Adding info for song 42 out of 100... 

	 Adding info for song 43 out of 1

	 Adding info for song 13 out of 100... 

	 Adding info for song 14 out of 100... 

	 Adding info for song 15 out of 100... 

	 Adding info for song 16 out of 100... 

	 Adding info for song 17 out of 100... 

	 Adding info for song 18 out of 100... 

	 Adding info for song 19 out of 100... 

	 Adding info for song 20 out of 100... 

	 Adding info for song 21 out of 100... 

	 Adding info for song 22 out of 100... 

	 Adding info for song 23 out of 100... 

	 Adding info for song 24 out of 100... 

	 Adding info for song 25 out of 100... 

	 Adding info for song 26 out of 100... 

	 Adding info for song 27 out of 100... 

	 Adding info for song 28 out of 100... 

	 Adding info for song 29 out of 100... 

	 Adding info for song 30 out of 100... 

	 Adding info for song 31 out of 100... 

	 Adding info for song 32 out of 100... 

	 Adding info for song 33 out of 100... 

	 Adding info for song 34 out of 100... 

	 Adding info for song 35 out of 100... 

	 Adding info for song 36 out of 1

## maybe there is a better way to do this but the following line isn't something that we had and therefore the dataframe I had exported previously only grabbed the last "songs_dfs" and was returned as a list instead of a dataframe

In [ ]:
playlist_df = pd.DataFrame(pd.concat(songs_dfs))

In [ ]:
playlist_df.head()

In [ ]:
playlist_df.to_csv('Resources/first50_playlists.csv')